In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

In [ ]:
X_train=pd.read_csv('../input/airline-passenger-satisfaction/train.csv')

In [ ]:
X_train.head()

In [ ]:
X_test=pd.read_csv('../input/airline-passenger-satisfaction/test.csv')

In [ ]:
X_test.head(10)

In [ ]:
X_train.columns

In [ ]:
corr=X_train.corr()
corr_d=corr['Departure Delay in Minutes'].sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(11,10))
sns.heatmap(corr,linewidths=0.6,annot=True,fmt='.1f',cmap='coolwarm',square=True,robust=True)

In [ ]:
#checking relation between departure and arrival delay.
plt.figure(figsize=(9,7))
sns.scatterplot('Departure Delay in Minutes','Arrival Delay in Minutes',data=X_train,color='g',edgecolor='none')

In [ ]:
X_train.dropna(axis=0,subset=['satisfaction'],inplace=True)
X_test.dropna(axis=0,subset=['satisfaction'],inplace=True)

In [ ]:
plt.figure(figsize=(11,7))
plt.subplot(1,2,1)
sns.countplot('Inflight service',data=X_train,hue='Customer Type')
plt.legend(loc='upper left')
plt.subplot(1,2,2)
sns.countplot('Cleanliness',data=X_train,hue='Customer Type')
plt.legend(loc='upper left')
plt.tight_layout()

In [ ]:
#here we check variation in ratings,prominently people of Business class have given good 
#feedback compared to eco and Eco plus classes.

plt.figure(figsize=(11,7))
plt.subplot(1,2,1)
sns.countplot('On-board service',data=X_train,hue='Class')
plt.legend(loc='upper left')
plt.ylabel('Count',fontsize=12)
plt.subplot(1,2,2)
sns.countplot('Leg room service',data=X_train,hue='Class')
plt.legend(loc='upper left')
plt.ylabel('Count',fontsize=12)
plt.tight_layout()

In [ ]:
#from here we can infer that mostly Business class people have been priortized followed by Eco class with
#almost same importance.

plt.figure(figsize=(11,7))
plt.subplot(1,2,1)
sns.countplot('Inflight entertainment',data=X_train,hue='Class')
plt.subplot(1,2,2)
sns.countplot('Checkin service',data=X_train,hue='Class')
plt.legend(loc='upper left')
plt.tight_layout()

In [ ]:
y_train=X_train.satisfaction
X_train=X_train.drop('satisfaction',axis=1)
y_test=X_test.satisfaction
X_test=X_test.drop('satisfaction',axis=1)

In [ ]:
#preprocess the data 
from category_encoders import CountEncoder
ce=CountEncoder()
ce_train=pd.DataFrame(ce.fit_transform(X_train,y_train))
ce_test=pd.DataFrame(ce.transform(X_test,y_test))

In [ ]:
ce_train=ce_train.fillna(0)
ce_test=ce_test.fillna(0)

### Train the model

In [ ]:
from catboost import CatBoostClassifier
cat=CatBoostClassifier(learning_rate=0.09)
cat.fit(ce_train,y_train)
preds=cat.predict(ce_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,preds))

### Confusion Matrix 

In [ ]:
matrix=confusion_matrix(y_test,preds)
plt.figure(figsize=(8,6))
sns.heatmap(matrix,annot=True,fmt='0.2f')